In [8]:
class DataMemory:

    def __init__(self, initFile: dict):
        self.data = dict() 
        for addr, data in initFile.items():
            if addr.startswith("0x"):
                self.data[int(addr, 16)] = data
            else:
                self.data[int(addr)] = data
    
    def read(self, addr: int) -> int:
        if self.data.get(addr) is None:
            return 0
        else:
            return self.data.get(addr)

    def write(self, addr: int, data: int) -> int:
        self.data[addr] = data

In [9]:
dataMemory = DataMemory({})

In [15]:
#Testing Datamemory
for i in range(512):
    dataMemory.write(i,i);
for i in range(512):
    print(dataMemory.read(i));

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
state = []

##################################################################
class VLIW:
    # Visible Architecture State.
    def __init__(self):
        self.PC = 0
        self._PC = 0
        self.RRB = 0
        self._RRB = 0
        self.LC = 0
        self._LC = 0
        self.EC = 0
        self._EC = 0
        self.PhysicalRegisterFile = [i for i in range(96)]
        self._PhysicalRegisterFile = []
        self.PredicateRegisters = [False for _ in range(96)]
        self._PredicateRegisters = []

        # Functional pipelines
        self.ALU0Pipe = {
            "predicate": False,
            "opcode": "alu", # alu, updateLC, updateEC, updateRRB, updatePredicate
            "targetReg": 0,
            "value": 0
        }

        self.ALU1Pipe = {
            "predicate": False,
            "opcode": "",
            "targetReg": 0,
            "value": 0
        }

        self.BranchPipe = {
            "predicate": False,
            "opcode": "hw", # lw or loop
            "targetPC": 0,
        }

        self.MemoryPipe = {
            "predicate": False,
            "opcode": "load", # load or store
            "address": 0,
            "data": 0,
            "loadDestReg": 0,
        }

        self.MultiplierPipe = [
            {
                "predicate": False,
                "targetReg": 0,
                "result": 0,
            },
            {
                "predicate": False,
                "targetReg": 0,
                "result": 0,
            },
            {
                "predicate": False,
                "targetReg": 0,
                "result": 0,
            }
        ]

    def serialize(self) -> dict:
        return {
            "PC": self.PC,
            "RRB": self.RRB,
            "LC": self.LC,
            "EC": self.EC,
            "PhysicalRegisterFile": self.PhysicalRegisterFile.copy(),
            "PredicateRegisters": self.PredicateRegisters.copy(),
            "ALU0": self.ALU0Pipe.copy(),
            "ALU1": self.ALU1Pipe.copy(),
            "Branch": self.BranchPipe.copy(),
            "Memory": self.MemoryPipe.copy(),
            "Multiply": self.MultiplierPipe.copy(),
            "MemoryData": dataMemory.data.copy()
        }

    _debug_currentCycleUpdate = []

    def updateRegister(self, name: str, value: int):
        if name in self._debug_currentCycleUpdate:
            print("Warning: Multiple instructions are updating the register {}.".format(name))
        else:
            self._debug_currentCycleUpdate.append(name)
        
        if name.startswith("x"):
            self.PhysicalRegisterFile[int(name[1:])] = value
        elif name.startswith("p"):
            self.PredicateRegisters[int(name[1:])] = value != 0
        elif name == "LC":
            self.LC = value
        elif name == "RRB":
            self.RRB = value
        elif name == "EC":
            self.EC = value
        else:
            assert False, "Wrong register name is met: {}".format(name)

    def renameRegister(self, idx: int) -> int:

        assert idx >= 0 and idx < 96, "Trying to rename a register out of the specific range."

        if idx >= 32:
            potential = idx - self.RRB
            if potential < 32:
                return potential + 64
            return potential
        
        return idx

    def parseImmediate(self, i: str) -> int:
        if i.startswith("0x"):
            return int(i, 16) # at present we only support hex and decimal.
        return int(i)

    def parse(self, i: str) -> dict:
        # i could be a format like "(pX) inst dst, src, third"
        # this function is trying to separate the predication.
        info = i.split()
        predication = True
        info[0] = info[0].strip()
        if info[0].startswith("(") and info[0].endswith(")"):
            # with predication
            idx = int(info[0][2:-1].strip())
            assert idx <= 95, "Undefined predicate register: p{}".format(idx)
            predication = self.PredicateRegisters[self.renameRegister(idx)]
            info.remove(info[0])
        
        operands: list[str] = []

        for item in info[1:]:
            item = item.strip()
            if item.endswith(","):
                operands.append(item[:-1].strip())
            else:
                operands.append(item.strip())

        return {
            "predicate": predication,
            "opcode": info[0].strip(),
            "operands": operands
        }

    def decodeALUInstruction(self, i: str) -> dict:
        decoded = self.parse(i)
        predication: bool = decoded["predicate"]
        opcode: str = decoded["opcode"]
        ops: list[str] = decoded["operands"]
        
        # Now start the normal instruction decoding.
        assert opcode in ["add", "addi", "sub", "mov", "nop"], "Undefined instruction: {}".format(i)
        
        # classify instruction by its type
        if opcode in ["add", "addi", "sub"]:
            # define the source
            assert ops[0].startswith('x'), "Cannot determine the destination: {}".format(i)
            dest =  self.renameRegister(int(ops[0][1:]))

            assert ops[1].startswith('x'), "Cannot determine the source reg: {}".format(i)
            src1 = self.renameRegister(int(ops[1][1:]))

            if opcode == "addi":
                src2 = int(ops[2])
            else:
                assert ops[2].startswith('x'), "Cannot determine the 2nd source reg: {}".format(i)
                src2 = self.renameRegister(int(ops[2][1:]))

            result = 0
            if opcode == "add":
                result = self.PhysicalRegisterFile[src1] + self.PhysicalRegisterFile[src2]
            elif opcode == "addi":
                result = self.PhysicalRegisterFile[src1] + src2
            elif opcode == "sub":
                result = self.PhysicalRegisterFile[src1] - self.PhysicalRegisterFile[src2]
                if result < 0:
                    result = result + 0x10000000000000000 # 2-complementary
            result = result & 0xFFFFFFFFFFFFFFFF

            return {
                "predicate": predication,
                "opcode": "alu", # alu, updateLC, updateEC, updateRRB, updatePredicate
                "targetReg": dest,
                "value": result
            }

        elif opcode == "mov":
            if ops[0].startswith("p"):
                # it's updating a predicate
                idx = self.renameRegister(int(ops[0][1:]))

                if ops[1] == "true":
                    return {
                        "predicate": predication,
                        "opcode": "updatePredicate", # alu, updateLC, updateEC, updateRRB, updatePredicate
                        "targetReg": idx,
                        "value": 1
                    }
                elif ops[1] == "false":
                    return {
                        "predicate": predication,
                        "opcode": "updatePredicate", # alu, updateLC, updateEC, updateRRB, updatePredicate
                        "targetReg": idx,
                        "value": 0
                    }
                else:
                    assert False, "Cannot determine the source operand: {}".format(i)
            elif ops[0].upper() in ["LC", "EC", "RRB"]:
                dest = ops[0].upper()
                value = int(ops[1])
                if ops[0].upper() == "RRB":
                    assert value < 64, "The maximum value of RRB is 63. The value you provide causes overflow."
                return {
                    "predicate": predication,
                    "opcode": "update{}".format(dest), # alu, updateLC, updateEC, updateRRB, updatePredicate
                    "targetReg": 0,
                    "value": value
                }
            elif ops[0].startswith('x'):
                dst = self.renameRegister(int(ops[0][1:]))
                if ops[1].startswith('x'):
                    src = self.renameRegister(int(ops[1][1:]))
                    return {
                        "predicate": predication,
                        "opcode": "alu",
                        "targetReg": dst,
                        "value": self.PhysicalRegisterFile[src]
                    }
                else: # It should be an integer
                    return {
                        "predicate": predication,
                        "opcode": "alu",
                        "targetReg": dst,
                        "value": self.parseImmediate(ops[1])
                    }
            else:
                assert False, "Unknown instruction: {}".format(i)
        elif opcode == "nop":
            return {
                "predicate": False,
                "opcode": "alu", # alu, updateLC, updateEC, updateRRB, updatePredicate
                "targetReg": 0,
                "value": 0
            }

    def decodeMultiplierInstruction(self, i: str) -> dict:
        decoded = self.parse(i)
        predication: bool = decoded["predicate"]
        opcode: str = decoded["opcode"]
        ops: list[str] = decoded["operands"]

        assert opcode in ["mulu", "nop"]

        if opcode == "mulu":
            assert ops[0].startswith('x') and ops[1].startswith('x') and ops[2].startswith('x'), "Undefined instruction: {}".format(i)
            dest = self.renameRegister(int(ops[0][1:]))
            src1 = self.renameRegister(int(ops[1][1:]))
            src2 = self.renameRegister(int(ops[2][1:]))
            return {
                "predicate": predication,
                "targetReg": dest,
                "result": (self.PhysicalRegisterFile[src1] * self.PhysicalRegisterFile[src2]) & 0xFFFFFFFFFFFFFFFF,
            }
        else:
            return {
                "predicate": False,
                "targetReg": 0,
                "result": 0,
            }


    def decodeLoadStoreInstruction(self, i: str) -> dict:
        decoded = self.parse(i)
        predication: bool = decoded["predicate"]
        opcode: str = decoded["opcode"]
        ops: list[str] = decoded["operands"]

        assert opcode in ["ld", "st", "nop"]

        if opcode == "nop":
            return {
                "predicate": False,
                "opcode": "load", # load or store
                "address": 0,
                "data": 0,
                "loadDestReg": 0,
            }

        # ops[0]: xNN
        # ops[1]: imm(xMM)
        assert ops[0].startswith('x') and "(" in ops[1] and ")" in ops[1], "Undefined instruction: {}".format(i)

        dest = self.renameRegister(int(ops[0][1:]))
        imm = ops[1].split("(")[0].strip()
        if len(imm) == 0:
            imm = 0
        else:
            imm = int(imm)

        add = ops[1].split("(")[1].strip()[1:-1]
        addr = self.PhysicalRegisterFile[self.renameRegister(int(add))] + imm

        if opcode == "ld":
            return {
                "predicate": predication,
                "opcode": "load", # load or store
                "address": addr,
                "data": 0,
                "loadDestReg": dest,
            }
        elif opcode == "st":
            return {
                "predicate": predication,
                "opcode": "store", # load or store
                "address": addr,
                "data": self.PhysicalRegisterFile[dest],
                "loadDestReg": 0,
            }
            


    def decodeBrancInstruction(self, i: str) -> dict:
        decoded = self.parse(i)
        predication: bool = decoded["predicate"]
        opcode: str = decoded["opcode"]
        ops: list[str] = decoded["operands"]

        assert opcode in ["loop", "loop.pip", "nop"], "Undefined instruction: {}".format(i)

        if opcode == "loop":
            return {
                "predicate": predication,
                "opcode": "loop", # lw or loop
                "targetPC": int(ops[0]),
            }
        elif opcode == "loop.pip":
            return {
                "predicate": predication,
                "opcode": "hw", # lw or loop
                "targetPC": int(ops[0]),
            }

        return {
            "predicate": False,
            "opcode": "hw", # lw or loop
            "targetPC": 0,
        }

    def tick(self):
        ## PC Propagate
        if self.PC >= len(instructionMemory):
            inst = ["nop", "nop", "nop", "nop", "nop"]
        else:
            inst = instructionMemory[self.PC]
        
        assert len(inst) == 5, "Each bundle should always have 5 instructions"
    
        # Branch Unit will be immediately updated, because its' combinational logic.
        #### inst[4] -> Branch
        self.BranchPipe = self.decodeBrancInstruction(inst[4])

        # record the state
        state.append(self.serialize())

        # Now start latch other data structures.
        ## Execution Stage
        self._debug_currentCycleUpdate.clear()
        
        #### ALUs
        for aluPipe in [self.ALU0Pipe, self.ALU1Pipe]:
            if aluPipe["predicate"]:
                idx = aluPipe["targetReg"]
                value = aluPipe["value"]
                if aluPipe["opcode"] == "alu":
                    self.updateRegister("x{}".format(idx), value)
                elif aluPipe["opcode"] == "updateLC":
                    self.updateRegister("LC", value)
                elif aluPipe["opcode"] == "updateEC":
                    self.updateRegister("EC", value)
                elif aluPipe["opcode"] == "updateRRB":
                    self.updateRegister("RRB", value)
                elif aluPipe["opcode"] == "updatePredicate":
                    self.updateRegister("p{}".format(idx), value)
                else:
                    assert False, "Wrong opcode is provided: {}".format(aluPipe["opcode"])
        
        #### Memory
        if self.MemoryPipe["predicate"]:
            if self.MemoryPipe["opcode"] == "load":
                self.updateRegister(
                    "x{}".format(self.MemoryPipe["loadDestReg"]),
                    dataMemory.read(self.MemoryPipe["address"])
                )
            elif self.MemoryPipe["opcode"] == "store":
                dataMemory.write(
                    self.MemoryPipe["address"],
                    self.MemoryPipe["data"]
                )
            else:
                assert False, "Wrong opcode"

        #### Multiplier: the most complex one.
        ##### Always pop the last one.
        if self.MultiplierPipe[2]["predicate"]:
            self.updateRegister(
                "x{}".format(self.MultiplierPipe[2]["targetReg"]),
                self.MultiplierPipe[2]["result"]
            )
        self.MultiplierPipe.pop()

        #### inst[0] -> ALU0
        self.ALU0Pipe = self.decodeALUInstruction(inst[0])
        #### inst[1] -> ALU1
        self.ALU1Pipe = self.decodeALUInstruction(inst[1])
        #### inst[2] -> MUL
        self.MultiplierPipe.insert(0, self.decodeMultiplierInstruction(inst[2]))
        #### inst[3] -> MEM
        self.MemoryPipe = self.decodeLoadStoreInstruction(inst[3])
        

        #### Branch Unit (It has zero latency.)
        if self.PC >= len(instructionMemory):
            self.PC = self.PC
        else:
            self.PC = self.PC + 1
        
        if self.BranchPipe["predicate"]:
            if self.BranchPipe["opcode"] == "loop":
                if self.LC > 0:
                    self.updateRegister("LC", self.LC - 1)
                    self.PC  = self.BranchPipe["targetPC"]
            elif self.BranchPipe["opcode"] == "hw":
                if self.LC > 0:
                    self.updateRegister("LC", self.LC - 1)
                    self.updateRegister("RRB", self.RRB + 1)
                    self.updateRegister("p{}".format(self.renameRegister(32)), 1)
                    self.PC = self.BranchPipe["targetPC"]
                elif self.EC > 0:
                    self.updateRegister("EC", self.EC - 1)
                    self.updateRegister("RRB", self.RRB + 1)
                    self.updateRegister("p{}".format(self.renameRegister(32)), 0)
                    self.PC = self.BranchPipe["targetPC"]
                else:
                    self.updateRegister("p{}".format(self.renameRegister(32)), 0)
        print(self.serialize())


In [12]:
instructionMemory = [
["mov LC, 100", "mov x33, 0x1000", "nop", "nop", "nop"],
["mov x42, 1", "mov x4, 25", "nop", "nop", "nop"],
["mov p32, true", "mov EC, 1", "nop", "nop", "nop"],
["(p32) addi x32, x33, 1", "nop", "nop", "(p32) ld x35, 0(x33)", "nop"],
["nop", "nop", "(p32) mulu x38, x35, x4", "(p33) st x39, 0(x34)", "nop"],
["nop", "nop", "(p32) mulu x41, x42, x35", "nop", "loop.pip 3"],
["nop", "nop", "nop", "st x42, 0(x33)", "nop"]
]

In [13]:
def simulate(instructionMemory_in):
    processor = VLIW()
    global instructionMemory
    global dataMemory
    global state
    instructionMemory = instructionMemory_in
    dataMemory = DataMemory({})
    state = []

    # In the main loop, let's see what happens
    while True:
        processor.tick()

        if processor.PC >= len(instructionMemory):
            # ok, now it's possible to see a stop. do two more cycles.
            processor.tick()
            processor.tick()
            break

    # Finally, dump the state to the file
    return dataMemory

In [14]:
DataMem=simulate(instructionMemory)

{'PC': 1, 'RRB': 0, 'LC': 0, 'EC': 0, 'PhysicalRegisterFile': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95], 'PredicateRegisters': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, 